#### Reference :
* https://www.kaggle.com/demetrypascal/better-train-csv-format-keras-starter
* https://www.kaggle.com/rsamdev/kaggle-starter-keras
* https://www.kaggle.com/arnabs007/apple-leaf-diseases-with-inceptionresnetv2-keras
* https://www.kaggle.com/emorkrin/baseline-keras-17-04/data#GoogLeNet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir('/kaggle/input/plant-pathology-2021-fgvc8/')

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet169, Xception, InceptionResNetV2, MobileNetV2, InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

plt.style.use('seaborn')
%matplotlib inline

In [ ]:
train_df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')
test_df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv')
print("Dataset Shape: ",train_df.shape)
train_df.head()

In [ ]:
train_df['labels'] = train_df['labels'].apply(lambda x: tuple(x.split(" ")))
train_df.head()

In [ ]:
datapath = glob('/kaggle/input/plant-pathology-2021-fgvc8/train_images/*')
print("Image Datasets Shape: ", len(datapath))

# EDA
* Labels per Value Percent
* Plot Sample Images

In [ ]:
f,ax = plt.subplots(1,1,figsize=(18,8))
ax = sns.countplot(train_df['labels'], order=train_df['labels'].value_counts().sort_values(ascending=False).index)
ax.set_xlim(0,8)
ax.set_title('Train Dataset per labels Count')
plt.show()

In [ ]:
train_df['labels'].value_counts()

In [ ]:
unique_list = np.unique(train_df['labels'])
print(unique_list)
print(train_df['labels'].value_counts().count())

In [ ]:
def read_image(path):
    path = os.path.join('../input/plant-pathology-2021-fgvc8/train_images',path["image"])
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile, dtype = tf.float32)
                  
    return image

In [ ]:
def get_label(path):
    return path['labels']

In [ ]:
def get_label_image(path):
    label = get_label(path)
    image = read_image(path)
    return label, image

In [ ]:
index_list = np.random.randint(128, size=5)
for i in index_list:
    path = train_df.iloc[i]
    sample_label, sample_image = get_label_image(path)
    plt.imshow(sample_image)
    plt.title(sample_label)
    plt.show()

# Feature Engineering
* Split Train / Validation Data
* Image Data Generate

In [ ]:
INPUT_SIZE = (256,256,3)
BATCH_SIZE = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.07, 
    zoom_range= 0.1, 
    horizontal_flip = True,
    validation_split=0.2
)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    train_df,
    directory="../input/plant-pathology-2021-fgvc8/train_images",
    x_col='image',
    y_col='labels',
    target_size=INPUT_SIZE[:2],
    subset='training'
)

val_generator = datagen.flow_from_dataframe(
    train_df,
    directory="../input/plant-pathology-2021-fgvc8/train_images",
    x_col='image',
    y_col='labels',
    target_size=INPUT_SIZE[:2],
    subset='validation'
)

In [ ]:
train_generator.class_indices

In [ ]:
for x, y in train_generator:
    break

In [ ]:
for i in range(5):
    plt.title(y[i])
    plt.imshow(x[i])
    plt.show()

# Modeling
* DenseNet 169

In [ ]:
weights_path = "../input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
pre_model = InceptionV3(include_top=False, weights=weights_path, input_shape=INPUT_SIZE)

In [ ]:
model = tf.keras.Sequential()
model.add(pre_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# you can use this callback func
callback= ReduceLROnPlateau(monitor='val_loss',
                       factor=.01, 
                       patience=3, 
                       min_lr=1e-5) 

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.00007),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator)//BATCH_SIZE,
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)//BATCH_SIZE
)

In [ ]:
f,ax = plt.subplots(2,1,figsize=(20,10))
ax[0].plot(history.history['loss'])
ax[0].plot(history.history['val_loss'])
ax[0].set_title('Model loss')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Val'], loc='upper left')

ax[1].plot(history.history['accuracy'])
ax[1].plot(history.history['val_accuracy'])
ax[1].set_title('Model accuracy')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Val'], loc='upper left')

f.tight_layout()
plt.show()

# Submission

In [ ]:
submission = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_generator =  datagen.flow_from_dataframe(
    submission,
    directory="../input/plant-pathology-2021-fgvc8/test_images",
    x_col='image',
    y_col=None,
    class_mode=None,
    target_size=INPUT_SIZE[:2]
)

In [ ]:
train_generator.class_indices.items()

In [ ]:
def get_key(val):
    for key, value in train_generator.class_indices.items():
        if val == value:
            return key

In [ ]:
preds = model.predict(test_generator)
print(preds)

In [ ]:
def get_preds_to_labels(preds):
    pred_lists = []
    for pred in preds:
        pred_list = []
        health = (pred>=0.4)
        
        # get healthy
        if health.sum()==0:
            label = 'healthy'
            pred_list.append(label)
            
        elif pred[2]>=0.5:
            label = 'healthy'
            pred_list.append(label)
            
        # get eles label
        else:
            for j, sub in enumerate(pred):
                if sub>=0.28:
                    label = get_key(j)
                    pred_list.append(label)
                               
        pred_lists.append(' '.join(pred_list))
    return pred_lists

In [ ]:
preds_list = get_preds_to_labels(preds)

In [ ]:
submission['labels'] = preds_list
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

#### THX to Reading My Notebook🌈